# Image https://hub.docker.com/r/bitnami/elasticsearch#!&_escaped_fragment_=

Ports: 9200, 9201, 9300
Mount: -v /path/to/elasticsearch-data-persistence:/bitnami/elasticsearch/data

curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_cluster/settings -d '{ "transient": { "cluster.routing.allocation.disk.threshold_enabled": false } }'

curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}


http://localhost:9200/?pretty

es5 container

Array mapping: https://stackoverflow.com/questions/67581594/elasticsearch-mapping-for-array

Habr guide: https://habr.com/ru/articles/280488/'

In [11]:
import requests

In [12]:
r = requests.put('http://localhost:9200/_cluster/settings', json={ "transient": { "cluster.routing.allocation.disk.threshold_enabled": False } })

In [16]:
r.json()

{'acknowledged': True,
 'persistent': {},
 'transient': {'cluster': {'routing': {'allocation': {'disk': {'threshold_enabled': 'false'}}}}}}

In [20]:
r = requests.put('http://localhost:9200/_all/_settings', json={"index.blocks.read_only_allow_delete": None})

In [21]:
r.json()

{'error': {'root_cause': [{'type': 'index_not_found_exception',
    'reason': 'no such index [_all] and no indices exist',
    'resource.type': 'index_or_alias',
    'resource.id': '_all',
    'index_uuid': '_na_',
    'index': '_all'}],
  'type': 'index_not_found_exception',
  'reason': 'no such index [_all] and no indices exist',
  'resource.type': 'index_or_alias',
  'resource.id': '_all',
  'index_uuid': '_na_',
  'index': '_all'},
 'status': 404}

In [22]:
import json
from elasticsearch import Elasticsearch

In [23]:
client = Elasticsearch([{"host": "127.0.0.1", "scheme": "http", "port": 9200}])

client.ping()

True

In [44]:
indexName = "master"
# client.indices.delete(index=indexName)
# client.indices.create(index=indexName)

ObjectApiResponse({'acknowledged': True})

In [45]:
analyzer_settings = {
  "settings": {
    "analysis": {
      "filter": {
        "ru_stop": {
          "type": "stop",
          "stopwords": "_russian_"
        },
        "ru_stemmer": {
          "type": "stemmer",
          "language": "russian"
        }
      },
      "analyzer": {
        "custom_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "ru_stop",
            "ru_stemmer"
          ]
        }
      }
    }
  }
}

In [46]:
mappings_master = {
  "mappings": {
    "properties": {
      "master_id": {
        "type": "keyword"
      },
      "master_desc": {
        "type": "text",
        "analyzer": "custom_analyzer"
      },
      "master_feedbacks": {
        "type": "text",
        "analyzer": "custom_analyzer",
        "fields": {
          "master_feedbacks_row": {
            "type": "text",
            "analyzer": "custom_analyzer",
          }
        }
      }
    }
  }
}


client.indices.create(index=indexName, 
                            # doc_type="diseases", 
                           # include_type_name="true", 
                           body={**analyzer_settings, **mappings_master}
                          )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'master'})

In [47]:
with open('../masters.json', 'r') as f:
    dataStore = json.load(f)

for data in dataStore:
    try:
        client.index(
            index=data["index"],
            # doc_type=data["doc_type"],
            id=data["id"],
            body=data["body"]
        )
    except Exception as e:
        print(e, end="")
print("ok")

ok


# Query 2 - masters

In [146]:
indexName = "master"
searchFrom = 0
searchSize = 10

# 2
# searchBody = {
#     "_source": ["name"],
#     "query": {
#         "simple_query_string": {
#             "query": 'стаж',
#             "fields": ["master_desc"]
#         }
#     },
#     "size": searchSize
# }

searchBody = {
  "_source": ["master_desc"], # True - all data
  "query": {
    "match": {
      "master_desc": "стаж"
    }
  },
  "size": searchSize
}

# searchBody = {}
result = client.search(index=indexName, body=searchBody)
# , body=searchBody, from_=1, size=5)

In [147]:
result

ObjectApiResponse({'took': 14, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 44, 'relation': 'eq'}, 'max_score': 0.7704413, 'hits': [{'_index': 'master', '_id': '247830', '_score': 0.7704413, '_source': {'master_desc': 'Лора Эдуардовна Быкова, Стаж Работы: 14 Л./Г..'}}, {'_index': 'master', '_id': '578414', '_score': 0.7704413, '_source': {'master_desc': 'Терентий Валентинович Бобылев, Стаж Работы: 10 Л./Г..'}}, {'_index': 'master', '_id': '435930', '_score': 0.7704413, '_source': {'master_desc': 'Шарапова Раиса Владимировна, Стаж Работы: 8 Л./Г..'}}, {'_index': 'master', '_id': '111767', '_score': 0.7704413, '_source': {'master_desc': 'Семенов Родион Евсеевич, Стаж Работы: 12 Л./Г..'}}, {'_index': 'master', '_id': '756017', '_score': 0.7704413, '_source': {'master_desc': 'Макаров Родион Зиновьевич, Стаж Работы: 15 Л./Г..'}}, {'_index': 'master', '_id': '354808', '_score': 0.7704413, '_source': {'master_desc': 'Глаф

In [148]:
result['hits']

{'total': {'value': 44, 'relation': 'eq'},
 'max_score': 0.7704413,
 'hits': [{'_index': 'master',
   '_id': '247830',
   '_score': 0.7704413,
   '_source': {'master_desc': 'Лора Эдуардовна Быкова, Стаж Работы: 14 Л./Г..'}},
  {'_index': 'master',
   '_id': '578414',
   '_score': 0.7704413,
   '_source': {'master_desc': 'Терентий Валентинович Бобылев, Стаж Работы: 10 Л./Г..'}},
  {'_index': 'master',
   '_id': '435930',
   '_score': 0.7704413,
   '_source': {'master_desc': 'Шарапова Раиса Владимировна, Стаж Работы: 8 Л./Г..'}},
  {'_index': 'master',
   '_id': '111767',
   '_score': 0.7704413,
   '_source': {'master_desc': 'Семенов Родион Евсеевич, Стаж Работы: 12 Л./Г..'}},
  {'_index': 'master',
   '_id': '756017',
   '_score': 0.7704413,
   '_source': {'master_desc': 'Макаров Родион Зиновьевич, Стаж Работы: 15 Л./Г..'}},
  {'_index': 'master',
   '_id': '354808',
   '_score': 0.7704413,
   '_source': {'master_desc': 'Глафира Эльдаровна Потапова, Стаж Работы: 13 Л./Г..'}},
  {'_index